In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [2]:
# Read local data and concatenate it into one dataset
df_interfax = pd.read_csv(r'C:\Users\Sergey\Project\Disk\interfax_parsed_data.csv', index_col=0)

df_smart_labs = pd.read_csv(r'C:\Users\Sergey\Project\Disk\smart_labs_parced_data.csv')
df_smart_labs.rename(columns={'key_words': 'tags'}, inplace=True)

df_ria = pd.read_csv(r'C:\Users\Sergey\Project\Disk\ria_parsed_data.csv', index_col=0)

df_kommersant = pd.read_csv(r'C:\Users\Sergey\Project\Disk\kommersant_parced_data.csv')
df_kommersant.rename(columns={'key_words': 'tags'}, inplace=True)

df = pd.concat(
    [df_interfax, df_smart_labs, df_ria, df_kommersant],
    axis=0, 
    ignore_index=True
)

In [3]:
dbname = 'defaultdb'
user = 'doadmin'
password = 'AVNS_pWw7oK1YPADBn-TRPqp'
host = 'db-postgresql-fra1-12797-do-user-8775628-0.c.db.ondigitalocean.com'
port = '25060'

conn_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

In [6]:
db = create_engine(conn_string) 
conn = db.connect() 
conn.autocommit = True
df.to_sql('raw_data', con=conn, if_exists='replace', index=False) 
conn.close()

In [ ]:
%%time

db = create_engine(conn_string) 
conn = db.connect() 
conn.autocommit = True

gen = pd.read_sql_query('SELECT * FROM raw_data LIMIT 10000', conn, chunksize=100)
df = pd.concat([next(gen) for i in range(100)]).reset_index()
df = df.drop('id', axis=1)

conn.close()

In [ ]:
df